In [ ]:
import pandas as pd

import numpy as np

import datetime


In [ ]:
df = pd.read_table('finaldataset.txt', header=0,names = ['stop_tag', 'stop_name', 'time', 'bus_id', 'job_id', 'route'])

In [18]:
df = df.sort_values(by='time')
#df = df[df.stop_tag != 'U']
#df = df[df.stop_tag == 'fersstat_ob']

#df.head(50)
df.head(5)

,stop_tag,stop_name,time,bus_id,job_id,route,weekday,hour
906474,14thstat,14th St & State St,2017-08-01 06:43:42,491,401,green,2,0
906488,gcat,GCATT,2017-08-01 06:44:07,491,401,green,2,0
906502,glc,GLC,2017-08-01 06:48:38,491,401,green,2,0
906516,bakebldg,Baker Bldg,2017-08-01 06:49:33,491,401,green,2,0
906530,fersstat_ob,Ferst Dr & State St,2017-08-01 06:50:08,491,401,green,2,0


In [ ]:
# new_col = list(map(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"), df.time))
# df.assign(datetime = new_col)




In [ ]:
h1, m1 = (15, 30)
h2, m2 = (18, 30)
weekday = 2

def is_the_choosen_one(x):
    dt = datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
    tm = dt.time()
    wd = dt.isoweekday()
    if weekday != wd:
        return False
    if tm > datetime.time(h2, m2):
        return False
    if tm < datetime.time(h1, m1):
        return False
    return True
        
#is_the_choosen_one('2018-04-24 16:43:42', 15, 30, 18, 30, 2)


In [ ]:
filter_bool = list(map(is_the_choosen_one, df.time))

df = df[filter_bool]

In [ ]:
print(len(df))


In [ ]:
df.groupby(['route', 'bus_id']).count().head(10)

In [ ]:
from collections import defaultdict

In [ ]:
def list_default_dict():
    return defaultdict(list)

times_id_stops = defaultdict(list_default_dict)

for name, group in df.groupby(['route', 'bus_id']):
    #print(name)
    route = name[0]
    bus_id = name[1]
    #if name == ('blue', 431):
        #print(group.head(20))
    for index, row in group.iterrows():
        stop_tag = row['stop_tag']
        atime = row['time']
        times_id_stops[route][bus_id].append((stop_tag, atime))
            
#times_id_stops['tech'][431]

In [ ]:
times_id_stops['tech'][431]

In [ ]:
def new_list_default_dict():
    return defaultdict(list_default_dict)

times_between_stops = defaultdict(new_list_default_dict)

def add_times_to_megadict(megadict, times_id_stops, route, bus_id):
    bus_times = times_id_stops[route][bus_id]
    L = len(bus_times)
    for k in range(L):
        w = 1
        stop_tag, actual_time = bus_times[k]
        seguir = True
        while seguir and k+w <= L-1:
            next_stop_tag, next_time = bus_times[k+w]
            if stop_tag == next_stop_tag:
                seguir = False
            t1 = datetime.datetime.strptime(actual_time, "%Y-%m-%d %H:%M:%S")
            t2 = datetime.datetime.strptime(next_time, "%Y-%m-%d %H:%M:%S")
            travel_time = (t2-t1).total_seconds()/60
            #print(stop_tag, next_stop_tag, t2, t1, travel_time)

            if travel_time < 60:
                megadict[route][stop_tag][next_stop_tag].append(travel_time)
            w += 1
            if w >= L or travel_time > 60:
                seguir = False
            if not seguir:
                w = 1

                
add_times_to_megadict(times_between_stops, times_id_stops, 'blue', 439)                
times_between_stops['blue']['fersatla']['8thhemp']
    
    




In [ ]:
import time
st = time.time()
times_between_stops = defaultdict(new_list_default_dict)
for name, group in df.groupby(['route', 'bus_id']):
    route, bus_id = name
    add_times_to_megadict(times_between_stops, times_id_stops, route, bus_id)                
ft = time.time() - time.time()
print(ft, 'seconds')



In [ ]:
# import pickle

# with open('times_between_stops.pickle', 'wb') as handle:
#     pickle.dump(times_between_stops, handle)

In [ ]:
#times_between_stops['blue']['fersatla']['8thhemp']

new_mega_dict = {}
for key, value in times_between_stops.items():
    dict1 = {}
    for key2, value2 in value.items():
        dict2 = {}
        for key3, value3 in value2.items():
            dict2[key3] = value3
        dict1[key2] = dict2
    new_mega_dict[key] = dict1

In [ ]:
# import pickle

# with open('new_mega_dict_t1518.pickle', 'wb') as handle:
#     pickle.dump(new_mega_dict, handle)

In [ ]:

# stop1 = 'studcentr'
# stop2 = 'fersatla'
# plt.figure()

# print(len(times_between_stops['blue'][stop1][stop2]), 'samples')

# print('95% p', np.percentile(times_between_stops['blue'][stop1][stop2], 95))


# plt.title("Travel time distribution between {} and {}".format(stop1, stop2))
# plt.hist(times_between_stops['blue'][stop1][stop2], bins=10, density=True,  edgecolor='black', linewidth=1)
# plt.show()

In [ ]:
from matplotlib import pyplot as plt
import datetime
%matplotlib notebook


def list_default_dict():
    return defaultdict(list)


arrival_time = defaultdict(list_default_dict)

for name, group in df.groupby(['route', 'stop_tag']):
    route, stop_tag = name
    #print(route, stop_tag)
    arrival_time[route][stop_tag] = list(group['time'])
    
#print(arrival_time['blue']['studcentr'])
print(len(arrival_time['blue']['studcentr']))



time_distribution = []
for k in range(len(arrival_time['blue']['studcentr'])-1):
    t1 = arrival_time['blue']['studcentr'][k]
    t1 = datetime.datetime.strptime(t1, "%Y-%m-%d %H:%M:%S")
    t2 = arrival_time['blue']['studcentr'][k+1]
    t2 = datetime.datetime.strptime(t2, "%Y-%m-%d %H:%M:%S")
    
    head_way = (t2-t1).total_seconds()/60
    if head_way < 60:
        time_distribution.append(head_way)
        
plt.hist(time_distribution, bins=12, density=True,  edgecolor='black', linewidth=1)
plt.show()

    
    


In [ ]:
def list_default_dict():
    return defaultdict(list)


arrival_time = defaultdict(list_default_dict)

for name, group in df.groupby(['route', 'stop_tag']):
    route, stop_tag = name
    #print(route, stop_tag)
    arrival_time[route][stop_tag] = list(group['time'])
    
#print(arrival_time['blue']['studcentr'])
print(len(arrival_time['blue']['studcentr']))




        

In [ ]:
from matplotlib import cm


X = []
Y = []
Z = []
col = []

route = 'tech'
stop = 'techsqua'
zero =  datetime.time(0, 0)
for k in range(len(arrival_time[route][stop])-1):
    t1 = arrival_time[route][stop][k]
    t1 = datetime.datetime.strptime(t1, "%Y-%m-%d %H:%M:%S")
    t2 = arrival_time[route][stop][k+1]
    t2 = datetime.datetime.strptime(t2, "%Y-%m-%d %H:%M:%S")
    
    head_way = (t2-t1).total_seconds()/60
    if head_way < 60:
        Z.append(t2.time())
        X.append(head_way)
        Y.append(t2.date())
        abstime = datetime.datetime.combine(datetime.date.min, t2.time()) - datetime.datetime.combine(datetime.date.min, zero)
        col.append(abstime.total_seconds())



plt.figure()
plt.scatter(Y, X, marker='s', alpha=1, c=col, s=1, cmap=cm.jet)
plt.xticks(rotation=90)
plt.show()

plt.figure()

plt.scatter(Y, Z, marker='s', alpha=1, c=X, s=1, cmap=cm.jet)
plt.xticks(rotation=90)
plt.show()




In [ ]:
df.groupby(['route', 'stop_tag']).count()

In [19]:
for name, group in df.groupby(['route', 'stop_tag']):
    print(name)

('blue', '3rdtech')
('blue', '4thtech')
('blue', '5thtech')
('blue', '8thhemp')
('blue', '8thwvil')
('blue', 'cherfers')
('blue', 'fersatla')
('blue', 'fersfowl')
('blue', 'fershemp_ob')
('blue', 'fershub')
('blue', 'fersklau')
('blue', 'fersstat')
('blue', 'fitthall_a')
('blue', 'naveapts_a')
('blue', 'reccent')
('blue', 'studcentr')
('blue', 'technorth')
('emory', 'clifgate')
('emory', 'clifgate_e')
('emory', 'clifwesl_e')
('emory', 'clifwesl_g')
('emory', 'fersatla_a')
('emory', 'fersatla_d')
('emory', 'hsrb_a')
('emory', 'hsrb_d')
('emory', 'mart_e')
('emory', 'marta_g')
('emory', 'piedbota_e')
('emory', 'piedbota_g')
('emory', 'woodmemo')
('green', '10thhemp')
('green', '14thbusy_a')
('green', '14thstat')
('green', 'bakebldg')
('green', 'creccent')
('green', 'fershemp')
('green', 'fershemp_ob')
('green', 'fersstat_ob')
('green', 'gcat')
('green', 'glc')
('green', 'hempcurr')
('green', 'ndec')
('green', 'reccent_ob')
('green', 'studcent')
('green', 'studcent_ib')
('green', 'tranhub

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

route = 'red'
stop1 = 'creccent'
stop2 = 'naveapts_a'


n_bins = 20

fig, axes = plt.subplots(nrows=1, ncols=1)
ax0 = axes

L80 = int((len(times_between_stops[route][stop1][stop2])*0.8)//1)


list80 = times_between_stops[route][stop1][stop2][:L80]
list20 = times_between_stops[route][stop1][stop2][L80:]

x_multi = [list80, list20]
ax0.hist(x_multi, n_bins, histtype='bar', density=True, label=['First 80%', "Last 20%"])
ax0.legend(prop={'size': 10})
ax0.set_title('80/20 test "{}"-"{}" | {} | {}:{}-{}:{}'.format(stop1, stop2, route, h1, m1, h2, m2))
plt.xlabel("travel time (minutes)")
plt.ylabel("Probability")

print(L80/0.8)
plt.show()

In [ ]:
print(np.average(list80), np.average(list20))
print(np.std(list80), np.std(list20))

In [ ]:
cdf20 = [k/len(list20) for k in range (1, len(list20)+1)]
cdf80 = [k/L80 for k in range(1, L80+1)]

list80.sort()
list20.sort()



plt.figure()


plt.plot(list80, cdf80, label='First80%')
plt.plot(list20, cdf20, label = 'Last 20%')



axes = plt.gca()

axes.legend(prop={'size': 10})


plt.title('CDF 80/20 "{}"-"{}" | {} | {}:{}-{}:{}'.format(stop1, stop2, route, h1, m1, h2, m2))
plt.xlabel("travel time (minutes)")
plt.ylabel("Probability")



axes.set_ylim([0,1])

plt.show()





In [ ]:
df = pd.read_table('finaldataset.txt', header=0,names = ['stop_tag', 'stop_name', 'time', 'bus_id', 'job_id', 'route'])
df = df.sort_values(by='time')
df = df[df.stop_tag != 'U']

times_at_stops = defaultdict(list_default_dict)
for name, group in df.groupby(['route', 'stop_tag']):
    route = name[0]
    stop_tag = name[1]
    for index, row in group.iterrows():
        atime = datetime.datetime.strptime(row['time'], "%Y-%m-%d %H:%M:%S")
        times_at_stops[route][stop_tag].append(atime)

    




In [ ]:
def get_headeays(times):
    headways = []
    for k in range(1, len(times)):
        t1 = times[k-1]
        t2 = times[k]
        hw = (t2-t1).total_seconds()/60
        if hw < 90:
            headways.append(hw)
    return headways


get_headeays(times_id_stops['blue']['5thtech'])

In [ ]:
h1, m1 = (15, 30)
h2, m2 = (18, 30)
weekday = 2

def filter_date(x):
    tm = x.time()
    wd = x.isoweekday()
    if weekday != wd:
        return False
    if tm > datetime.time(h2, m2):
        return False
    if tm < datetime.time(h1, m1):
        return False
    return True

def exp_wait_time(hwlist):
    ewt = 0
    for hw in hwlist:
        ewt += hw * hw/2
    return ewt/sum(hwlist)

times_id_stops['blue']['5thtech']

In [ ]:
waiting_stats = {}


for name, group in df.groupby(['route', 'stop_tag']):
    route = name[0]
    stop_tag = name[1]
    
    if route not in waiting_stats.keys():
        waiting_stats[route] = {}
        
    waiting_stats[route][stop_tag] = {}
    
    times_rs = times_id_stops[route][stop_tag]
    for k in range(1,8):
        fdist = [x for x in times_rs if x.isoweekday() == k]
        #hwww = get_headeays(fdist)
        
        waiting_stats[route][stop_tag][k] = {}

        for h in range(6):
            h1 = [7,9,11,13,16,18,21][h] - 1
            m1 = 30
            h2 = [7,9,11,13,16,18,21][h+1]
            m2 = 30
            newflist = [x for x in fdist if (x.time() > datetime.time(h1, m1) and x.time() < datetime.time(h2, m2))]
            hw = get_headeays(newflist)
            if hw != []:
                timewindow = '{}:{}-{}:{}'.format([7,9,11,13,16,18,21][h],'00',[7,9,11,13,16,18,21][h+1],'00')
                real = exp_wait_time(hw)
                ideal = np.mean(hw)/2
                waiting_stats[route][stop_tag][k][timewindow] = {
                    'Ewaitt': round(real,2),
                    'IdealWT': round(ideal,2),
                    'KPI': round(real/ideal -1,5)
                }
                print(timewindow,route, stop_tag, k, real/ideal -1, exp_wait_time(hw))
            else:
                waiting_stats[route][stop_tag][k][timewindow] = {
                    'Ewaitt': 0,
                    'IdealWT': 0,
                    'KPI': 0
                }
                
#print(waiting_stats)



In [ ]:
import pickle

with open('wtandkpis.pickle', 'wb') as handle:
    pickle.dump(waiting_stats, handle)

## Travel times

In [ ]:
import time
import pickle



times_between_stops = defaultdict(new_list_default_dict)
for name, group in df.groupby(['route', 'bus_id']):
    route, bus_id = name
    add_times_to_megadict(times_between_stops, times_id_stops, route, bus_id)                

new_mega_dict = {}
for key, value in times_between_stops.items():
    dict1 = {}
    for key2, value2 in value.items():
        dict2 = {}
        for key3, value3 in value2.items():
            dict2[key3] = value3
        dict1[key2] = dict2
    new_mega_dict[key] = dict1
    

with open('new_mega_dict_t1518.pickle', 'wb') as handle:
    pickle.dump(new_mega_dict, handle)

    
    


In [10]:
from collections import defaultdict
import pickle
import pandas as pd
import datetime

In [22]:
def time_cat(time):
    if time < datetime.time(7, 30):
        return 0
    elif time < datetime.time(9, 0):
        return 1
    elif time < datetime.time(11, 0):
        return 2
    elif time < datetime.time(13, 0):
        return 3
    elif time < datetime.time(16, 0):
        return 4
    elif time < datetime.time(18, 0):
        return 5
    elif time < datetime.time(21, 0):
        return 6
    else:
        return 0

def list_default_dict():
    return defaultdict(list)

def new_list_default_dict():
    return defaultdict(list_default_dict)

def add_times_to_megadict(megadict, times_id_stops, route, bus_id):
    bus_times = times_id_stops[route][bus_id]
    L = len(bus_times)
    for k in range(L):
        w = 1
        stop_tag, actual_time = bus_times[k]
        seguir = True
        while seguir and k+w <= L-1:
            next_stop_tag, next_time = bus_times[k+w]
            if stop_tag == next_stop_tag:
                seguir = False
            t1 = datetime.datetime.strptime(actual_time, "%Y-%m-%d %H:%M:%S")
            t2 = datetime.datetime.strptime(next_time, "%Y-%m-%d %H:%M:%S")
            travel_time = (t2-t1).total_seconds()/60
            #print(stop_tag, next_stop_tag, t2, t1, travel_time)

            if travel_time < 80:
                megadict[route][stop_tag][next_stop_tag].append(travel_time)
            w += 1
            if w >= L or travel_time > 60:
                seguir = False
            if not seguir:
                w = 1
    

In [25]:
df = pd.read_table('finaldataset.txt', header=0,names = ['stop_tag', 'stop_name', 'time', 'bus_id', 'job_id', 'route'])
df = df.sort_values(by='time')
df = df[df.stop_tag != 'U']

new_col = list(map(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").isoweekday(), df.time))
df = df.assign(weekday = new_col)
new_col = list(map(lambda x: time_cat(datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").time()), df.time))
df= df.assign(hour = new_col)

df.head(5)



,stop_tag,stop_name,time,bus_id,job_id,route,weekday,hour
906474,14thstat,14th St & State St,2017-08-01 06:43:42,491,401,green,2,0
906488,gcat,GCATT,2017-08-01 06:44:07,491,401,green,2,0
906502,glc,GLC,2017-08-01 06:48:38,491,401,green,2,0
906516,bakebldg,Baker Bldg,2017-08-01 06:49:33,491,401,green,2,0
906530,fersstat_ob,Ferst Dr & State St,2017-08-01 06:50:08,491,401,green,2,0


In [27]:
for name, group in df.groupby(['weekday', 'hour']):
    wd, hour = name
    print(wd, hour)
    
    times_id_stops = defaultdict(list_default_dict)
    for name2, group2 in group.groupby(['route', 'bus_id']):
        route = name2[0]
        bus_id = name2[1]
        for index, row in group2.iterrows():
            stop_tag = row['stop_tag']
            atime = row['time']
            times_id_stops[route][bus_id].append((stop_tag, atime))
        
        
    times_between_stops = defaultdict(new_list_default_dict)
    for name3, group3 in group.groupby(['route', 'bus_id']):
        route, bus_id = name3
        add_times_to_megadict(times_between_stops, times_id_stops, route, bus_id)                

    new_mega_dict = {}
    for key, value in times_between_stops.items():
        dict1 = {}
        for key2, value2 in value.items():
            dict2 = {}
            for key3, value3 in value2.items():
                dict2[key3] = value3
            dict1[key2] = dict2
        new_mega_dict[key] = dict1
    with open('tt_{}_{}.pickle'.format(wd, hour), 'wb') as handle:
        pickle.dump(new_mega_dict, handle)
    
    

1 0
[3.25, 3.6, 2.6666666666666665, 2.35, 2.6666666666666665, 3.7666666666666666, 2.8333333333333335, 2.683333333333333, 4.166666666666667, 4.766666666666667, 3.433333333333333, 3.933333333333333, 3.6, 3.433333333333333, 4.933333333333334, 3.85, 3.8833333333333333, 3.8333333333333335, 3.55, 4.216666666666667, 4.0, 4.3, 4.85, 2.5166666666666666, 3.6666666666666665, 3.5833333333333335, 3.183333333333333, 3.183333333333333, 4.266666666666667, 4.166666666666667, 4.516666666666667, 4.85, 4.5, 3.5, 4.016666666666667, 3.3333333333333335, 3.6666666666666665, 3.183333333333333, 4.35, 4.1, 4.1, 16.116666666666667, 3.75, 3.5166666666666666, 3.85, 5.016666666666667, 4.25, 3.7666666666666666, 3.433333333333333, 3.1, 4.983333333333333, 2.966666666666667, 5.25, 2.8, 5.266666666666667, 2.933333333333333, 4.45, 3.816666666666667, 3.183333333333333, 8.683333333333334, 3.6, 3.5833333333333335, 4.216666666666667, 3.15, 3.55, 2.7666666666666666, 3.8833333333333333, 3.0, 3.066666666666667, 4.116666666666666

[4.583333333333333, 4.85, 4.5, 5.35, 4.35, 3.7666666666666666, 3.9166666666666665, 3.0166666666666666, 3.0833333333333335, 3.5166666666666666, 3.433333333333333, 2.85, 4.433333333333334, 4.75, 3.183333333333333, 4.016666666666667, 3.5, 4.85, 2.75, 5.85, 4.333333333333333, 4.35, 5.183333333333334, 3.1666666666666665, 3.933333333333333, 5.35, 4.75, 3.683333333333333, 5.433333333333334, 5.0, 5.0, 4.25, 4.25, 4.25, 4.183333333333334, 3.8333333333333335, 5.016666666666667, 3.75, 5.0, 4.1, 4.016666666666667, 4.666666666666667, 3.9166666666666665, 5.266666666666667, 3.7666666666666666, 3.433333333333333, 3.0166666666666666, 3.683333333333333, 4.016666666666667, 4.083333333333333, 4.6, 4.6, 3.75, 4.583333333333333, 4.266666666666667, 3.5166666666666666, 4.683333333333334, 3.6, 4.1, 3.35, 4.25, 4.933333333333334, 5.5, 5.35, 5.6, 5.083333333333333, 4.5, 5.716666666666667, 3.4166666666666665, 4.583333333333333, 3.8333333333333335, 3.6, 4.35, 3.9166666666666665, 4.183333333333334, 7.26666666666666

[4.433333333333334, 4.933333333333334, 3.25, 2.3333333333333335, 3.5166666666666666, 2.6, 2.5833333333333335, 2.85, 2.5833333333333335, 4.25, 7.516666666666667, 3.7666666666666666, 4.266666666666667, 3.0833333333333335, 4.766666666666667, 4.666666666666667, 3.683333333333333, 3.0166666666666666, 3.683333333333333, 3.85, 5.1, 3.533333333333333, 3.8333333333333335, 3.25, 5.016666666666667, 4.016666666666667, 4.433333333333334, 4.166666666666667, 4.266666666666667, 4.016666666666667, 4.416666666666667, 5.266666666666667, 3.5, 3.9166666666666665, 3.9166666666666665, 3.5, 4.233333333333333, 3.6666666666666665, 3.933333333333333, 4.183333333333334, 3.933333333333333, 3.683333333333333, 4.6, 3.433333333333333, 3.0166666666666666, 3.2666666666666666, 5.266666666666667, 4.35, 5.6, 5.1, 3.4166666666666665, 3.25, 3.35, 2.1666666666666665, 4.1, 4.85, 4.6, 3.6666666666666665, 4.266666666666667, 2.5, 2.6666666666666665, 3.85, 3.683333333333333, 2.933333333333333, 4.016666666666667, 3.766666666666666

[5.516666666666667, 4.5, 5.516666666666667, 5.333333333333333, 5.6, 4.266666666666667, 4.266666666666667, 3.35, 3.25, 3.183333333333333, 5.416666666666667, 3.933333333333333, 3.3333333333333335, 3.6666666666666665, 3.9166666666666665, 4.35, 4.766666666666667, 3.0833333333333335, 3.5166666666666666, 3.5, 3.5833333333333335, 3.933333333333333, 4.666666666666667, 4.016666666666667, 3.7666666666666666, 4.916666666666667, 4.1, 4.083333333333333, 4.7, 4.1, 4.083333333333333, 5.25, 4.6, 7.8, 2.6666666666666665, 3.966666666666667, 5.2, 4.983333333333333, 4.016666666666667, 4.366666666666666, 4.433333333333334, 5.016666666666667, 5.3, 4.583333333333333, 3.6166666666666667, 4.0, 4.0, 4.016666666666667, 3.35, 4.183333333333334, 5.433333333333334, 7.433333333333334, 6.266666666666667, 6.166666666666667, 6.366666666666666, 6.433333333333334, 5.766666666666667, 4.583333333333333, 5.166666666666667, 5.183333333333334, 5.083333333333333, 4.766666666666667, 3.7666666666666666, 3.4166666666666665, 3.766

[7.1, 6.266666666666667, 3.8333333333333335, 4.333333333333333, 3.7666666666666666, 3.4166666666666665, 5.516666666666667, 6.85, 4.5, 5.516666666666667, 5.1, 16.7, 4.35, 4.433333333333334, 2.75, 2.85, 9.35, 9.483333333333333, 5.016666666666667, 4.333333333333333, 4.683333333333334, 4.583333333333333, 6.1, 5.35, 3.9166666666666665, 4.55, 2.6666666666666665, 3.1, 3.7666666666666666, 4.6, 7.383333333333334, 3.5166666666666666, 3.2666666666666666, 3.35, 2.5833333333333335, 3.183333333333333, 5.433333333333334, 3.5, 5.433333333333334, 4.516666666666667, 3.6, 3.933333333333333, 2.3333333333333335, 7.35, 3.85, 4.85, 5.85, 2.9166666666666665, 3.0166666666666666, 14.616666666666667, 6.35, 4.6, 3.1, 3.3333333333333335, 4.35, 9.433333333333334, 5.183333333333334, 4.35, 4.333333333333333, 3.2666666666666666, 3.933333333333333, 6.083333333333333, 4.766666666666667, 4.85, 3.4166666666666665, 3.75, 4.0, 3.9166666666666665, 5.183333333333334, 3.0166666666666666, 3.8333333333333335, 4.6, 6.6, 5.1833333

...

[7.6, 4.1, 4.266666666666667, 3.1, 3.6, 4.433333333333334, 5.266666666666667, 6.166666666666667, 4.133333333333334, 5.166666666666667, 4.6, 4.416666666666667, 6.6, 4.5, 4.016666666666667, 3.8333333333333335, 11.083333333333334, 4.1, 7.45, 4.616666666666666, 4.916666666666667, 3.6666666666666665, 8.45, 5.333333333333333, 5.683333333333334, 3.3333333333333335, 7.183333333333334, 6.166666666666667, 4.6, 4.1, 5.1, 4.416666666666667, 4.683333333333334, 6.8, 7.916666666666667, 3.35, 2.75, 5.6, 3.433333333333333, 5.166666666666667, 3.75, 4.683333333333334, 5.866666666666666, 5.566666666666666, 4.583333333333333, 6.65, 6.2, 3.75, 3.7666666666666666, 4.683333333333334, 5.516666666666667, 5.85, 4.85, 6.85, 10.6, 4.766666666666667, 7.6, 16.133333333333333, 3.6666666666666665, 10.083333333333334, 6.25, 4.183333333333334, 4.683333333333334, 5.183333333333334, 4.5, 4.6, 6.683333333333334, 13.366666666666667, 5.266666666666667, 6.6, 11.35, 4.85, 6.433333333333334, 12.85, 3.85, 5.516666666666667, 4.35

[7.766666666666667, 4.6, 5.583333333333333, 5.1, 5.6, 4.516666666666667, 5.016666666666667, 4.6, 3.9166666666666665, 4.933333333333334, 3.683333333333333, 4.0, 5.933333333333334, 6.933333333333334, 4.333333333333333, 5.933333333333334, 4.333333333333333, 4.916666666666667, 4.433333333333334, 5.266666666666667, 5.35, 5.766666666666667, 5.6, 5.516666666666667, 5.35, 5.516666666666667, 4.85, 4.016666666666667, 4.6, 4.35, 7.433333333333334, 4.083333333333333, 4.083333333333333, 7.85, 3.6666666666666665, 3.85, 6.6, 5.1, 4.766666666666667, 6.516666666666667, 6.0, 5.183333333333334, 5.416666666666667, 4.516666666666667, 5.25, 5.1, 8.433333333333334, 5.65, 5.616666666666666, 6.783333333333333, 4.95, 5.283333333333333, 4.7, 4.016666666666667, 4.516666666666667, 4.6, 3.5, 4.516666666666667, 3.3333333333333335, 4.016666666666667, 6.683333333333334, 4.833333333333333, 36.65, 4.333333333333333, 4.6, 4.25, 5.083333333333333, 5.1, 9.016666666666667, 4.85, 11.033333333333333, 4.516666666666667, 4.85, 

[5.766666666666667, 7.6, 7.6, 5.183333333333334, 4.766666666666667, 5.35, 9.65, 4.4, 5.883333333333334, 5.016666666666667, 6.35, 3.933333333333333, 3.183333333333333, 8.966666666666667, 9.783333333333333, 7.766666666666667, 7.1, 5.933333333333334, 8.683333333333334, 6.683333333333334, 11.35, 4.766666666666667, 4.766666666666667, 3.933333333333333, 3.683333333333333, 5.933333333333334, 9.516666666666667, 7.95, 9.766666666666667, 5.666666666666667, 8.933333333333334, 4.583333333333333, 6.1, 5.183333333333334, 5.1, 5.183333333333334, 10.35, 7.816666666666666, 5.2, 4.85, 5.866666666666666, 4.183333333333334, 14.133333333333333, 6.7, 8.45, 7.016666666666667, 5.1, 4.083333333333333, 6.516666666666667, 4.0, 6.933333333333334, 4.933333333333334, 5.516666666666667, 5.516666666666667, 5.083333333333333, 6.016666666666667, 5.5, 4.75, 5.516666666666667, 6.183333333333334, 8.116666666666667, 7.35, 8.683333333333334, 7.35, 5.016666666666667, 4.083333333333333, 6.016666666666667, 6.683333333333334, 3

[6.5, 5.016666666666667, 5.733333333333333, 3.816666666666667, 4.416666666666667, 6.933333333333334, 4.0, 3.933333333333333, 4.6, 4.85, 4.3, 4.183333333333334, 4.75, 3.75, 7.183333333333334, 4.866666666666666, 8.183333333333334, 4.166666666666667, 4.683333333333334, 4.933333333333334, 5.016666666666667, 3.5, 5.1, 5.166666666666667, 5.0, 4.416666666666667, 6.35, 5.683333333333334, 6.183333333333334, 6.616666666666666, 4.1, 8.1, 5.766666666666667, 5.683333333333334, 5.25, 4.5, 4.1, 5.8, 4.816666666666666, 5.233333333333333, 4.666666666666667, 6.1, 5.816666666666666, 6.433333333333334, 4.433333333333334, 4.766666666666667, 4.416666666666667, 4.6, 4.433333333333334, 18.8, 5.016666666666667, 5.333333333333333, 5.083333333333333, 5.683333333333334, 5.333333333333333, 5.1, 3.933333333333333, 5.016666666666667, 5.183333333333334, 6.85, 5.266666666666667, 4.833333333333333, 4.666666666666667, 4.766666666666667, 5.266666666666667, 4.416666666666667, 4.916666666666667, 6.35, 5.6, 4.91666666666666

[6.083333333333333, 6.083333333333333, 5.0, 5.516666666666667, 10.183333333333334, 2.6666666666666665, 4.35, 2.0, 3.5166666666666666, 9.083333333333334, 3.6, 4.183333333333334, 6.766666666666667, 2.5833333333333335, 7.266666666666667, 5.266666666666667, 4.1, 4.866666666666666, 5.183333333333334, 7.766666666666667, 8.45, 4.583333333333333, 4.5, 3.8333333333333335, 3.0166666666666666, 3.5833333333333335, 6.183333333333334, 6.183333333333334, 9.433333333333334, 5.983333333333333, 6.483333333333333, 3.3333333333333335, 6.016666666666667, 3.9166666666666665, 2.9166666666666665, 2.9166666666666665, 3.2666666666666666, 6.333333333333333, 4.783333333333333, 8.183333333333334, 6.6, 7.35, 4.0, 14.35, 43.016666666666666, 4.683333333333334, 7.1, 4.916666666666667, 7.016666666666667, 10.85, 4.583333333333333, 5.266666666666667, 5.0, 4.833333333333333, 5.6, 5.333333333333333, 3.683333333333333, 11.366666666666667, 3.716666666666667, 5.766666666666667, 7.183333333333334, 3.6666666666666665, 7.6, 5.66

In [ ]:
datetime.datetime.now().isoweekday()